# Analysis of Taskmaster Database Portfolio Project

Create a data science portfolio project using the datasets from A Ridiculously Comprehensive Taskmaster Database (source: https://www.kaggle.com/datasets/sujaykapadnis/comedians-challenged-ridiculous-taskmaster-ds).

Best project wins.

Your time starts now.

## Taskmaster

*Taskmaster* is a British comedy panel television series where a group of five comedians, actors, and television personalities compete in a series of challenges, or 'tasks'. The tasks are moderated by the Taskmaster's Assistant, Alex Horne, and then the titular Taskmaster, Greg Davies, judges the tasks and awards points to each contestant on a 1-5 point scale. At the end of each series, the contestant with the most points is crowned champion and awarded a trophy depicting the head of the Taskmaster.

In addition to the regular series, the series includes the specials *Champion of Champions* and *New Years Treat*. *Champion of Champions* has the champions of the five most recent series return and compete against one another for the title of **champion of champions**. To date, three *Champion of Champions* specials have aired. After 25 regular series of *Taskmaster* and 5 *Champion of Champions* specials are aired, there are plans for a *Champion of Champions of Champions* mega special, where the 5 winners of the *Champion of Champions* face off against each other. The *New Years Treat* special is a special that began airing every New Years Day starting in 2021 where a new group of contestants try to perform the tasks and compete for points for a single episode rather than a full series.

## A Ridiculously Comprehensive Taskmaster Database

The database **A Ridiculously Comprehensive Taskmaster Database** was compiled by Reddit user u/alohamori and hosted as a Datasette instance (source: https://www.reddit.com/r/taskmaster/comments/13zb9nl/i_put_together_a_ridiculously_comprehensive/). The original link is broken, however, but Kaggle user Sujay Kapdanis later made the database available for use on Kaggle. It is based on the site taskmaster.info, where Karl Craven has compiled information on *Taskmaster* and its many international spin-offs.

**A Ridiculously Comprehensive Taskmaster Database** contains 22 .csv files featuring information on the *Taskmaster UK* series. These files are up-to-date only to the eighth episode of the 16th series of the show (air date: November 9, 2023), including the *Champion of Champions* and *New Years Treat* episodes up to that point. On Kaggle, it states that the dataset was last "Updated a year ago". Therefore, at the time of writing (December 2024), the data is incomplete. Additionally, the dataframe 'tasks.csv' is empty except for the column headers, which made it unusable for any data extraction for this project.

Because u/alohamori's original hyperlink is broken, it is unclear whether these or other instances of missing data come from that original database or are due to Kapdanis' transfer of the .csv files to Kaggle.

## Import database

In [1]:
import numpy as np
import pandas as pd

In [2]:
task_briefs = pd.read_csv('task_briefs.csv')
podcast = pd.read_csv('podcast.csv')
team_tasks = pd.read_csv('team_tasks.csv')
series_scores = pd.read_csv('series_scores.csv')
special_locations = pd.read_csv('special_locations.csv')
normalized_scores = pd.read_csv('normalized_scores.csv')
task_winners = pd.read_csv('task_winners.csv')
profanity = pd.read_csv('profanity.csv')
teams = pd.read_csv('teams.csv')
discrepancies = pd.read_csv('discrepancies.csv')
title_coiners = pd.read_csv('title_coiners.csv')
task_readers = pd.read_csv('task_readers.csv')
tasks = pd.read_csv('tasks.csv')
tasks_by_objective = pd.read_csv('tasks_by_objective.csv')
intros = pd.read_csv('intros.csv')
episode_scores = pd.read_csv('episode_scores.csv')
people = pd.read_csv('people.csv')
episodes = pd.read_csv('episodes (1).csv')
attempts = pd.read_csv('attempts.csv')
objectives = pd.read_csv('objectives.csv')
series = pd.read_csv('series.csv')
measurements = pd.read_csv('measurements.csv')

## Episodes won by series champions

At the end of each *Taskmaster* episode, the contestant with the most points for the episode is declared the winner and recieves all of the prizes brought in by the contestants during the first task.

How much of a factor does winning individual episodes play into winning the series? I will compile the number of episodes that were won by each series' winner by performing a merge between the **series** and **episodes** dataframes. First, I will start by cleaning and transforming each dataframe to only include information regarding the completed series, not including specials.

### Clean and transform 'Series' dataframe

The **series** dataframe features information on each series of *Taskmaster*, including the *New Years Treat* and *Champion of Champions* specials. This information includes the series number and name, the number of episodes per series, the series champion, the broadcast and filming start and end dates, the total number of points acrued throughout the series, and the total number of tasks performed.

In [3]:
series

,id,name,episodes,champion,champion_label,air_start,air_end,studio_start,studio_end,points,tasks,special,TMI
0,1,Series 1,6,4.0,Josh Widdicombe,2015-07-28,2015-09-01,2015-03-23,2015-03-25,436.0,32.0,0,1
1,2,Series 2,5,11.0,Katherine Ryan,2016-06-21,2016-07-19,NaN,NaN,417.0,28.0,0,2
2,3,Series 3,5,16.0,Rob Beckett,2016-10-04,2016-11-01,NaN,NaN,386.0,27.0,0,3
3,4,Series 4,8,22.0,Noel Fielding,2017-04-25,2017-06-13,NaN,NaN,668.0,46.0,0,4
4,5,Series 5,8,24.0,Bob Mortimer,2017-09-13,2017-11-01,2017-07-03,2017-07-06,631.0,44.0,0,5
5,-1,CoC,2,29.0,Josh Widdicombe,2017-12-13,2017-12-20,2017-11-20,2017-11-20,164.0,10.0,1,6
6,6,Series 6,10,35.0,Liza Tarbuck,2018-05-02,2018-07-04,2018-03-22,2018-03-28,837.0,57.0,0,7
7,7,Series 7,10,40.0,Kerry Godliman,2018-09-05,2018-11-07,2018-07-19,2018-07-25,816.0,56.0,0,8
8,8,Series 8,10,45.0,Lou Sanders,2019-05-08,2019-07-10,2019-03-21,2019-03-27,749.0,54.0,0,9
9,9,Series 9,10,49.0,Ed Gamble,2019-09-04,2019-11-06,2019-07-18,2019-07-24,766.0,51.0,0,10


In [4]:
series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              22 non-null     int64  
 1   name            22 non-null     object 
 2   episodes        22 non-null     int64  
 3   champion        20 non-null     float64
 4   champion_label  20 non-null     object 
 5   air_start       22 non-null     object 
 6   air_end         22 non-null     object 
 7   studio_start    15 non-null     object 
 8   studio_end      15 non-null     object 
 9   points          21 non-null     float64
 10  tasks           21 non-null     float64
 11  special         22 non-null     int64  
 12  TMI             22 non-null     int64  
dtypes: float64(3), int64(4), object(6)
memory usage: 2.4+ KB


The dataframe **series** includes the *New Years Treat* and *Champion of Champions* specials. I am only interested in the regular series of the show. Therefore, I will create a new dataframe labelled **regular_series** which includes all of the regular series (non-specials) of Taskmaster while excluding the specials.

In [5]:
# Remove Champion of Champions and New Years Treat special episodes from "series"
regular_series = series[series['id'] >= 0]
regular_series

,id,name,episodes,champion,champion_label,air_start,air_end,studio_start,studio_end,points,tasks,special,TMI
0,1,Series 1,6,4.0,Josh Widdicombe,2015-07-28,2015-09-01,2015-03-23,2015-03-25,436.0,32.0,0,1
1,2,Series 2,5,11.0,Katherine Ryan,2016-06-21,2016-07-19,NaN,NaN,417.0,28.0,0,2
2,3,Series 3,5,16.0,Rob Beckett,2016-10-04,2016-11-01,NaN,NaN,386.0,27.0,0,3
3,4,Series 4,8,22.0,Noel Fielding,2017-04-25,2017-06-13,NaN,NaN,668.0,46.0,0,4
4,5,Series 5,8,24.0,Bob Mortimer,2017-09-13,2017-11-01,2017-07-03,2017-07-06,631.0,44.0,0,5
6,6,Series 6,10,35.0,Liza Tarbuck,2018-05-02,2018-07-04,2018-03-22,2018-03-28,837.0,57.0,0,7
7,7,Series 7,10,40.0,Kerry Godliman,2018-09-05,2018-11-07,2018-07-19,2018-07-25,816.0,56.0,0,8
8,8,Series 8,10,45.0,Lou Sanders,2019-05-08,2019-07-10,2019-03-21,2019-03-27,749.0,54.0,0,9
9,9,Series 9,10,49.0,Ed Gamble,2019-09-04,2019-11-06,2019-07-18,2019-07-24,766.0,51.0,0,10
10,10,Series 10,10,57.0,Richard Herring,2020-10-15,2020-12-17,2020-07-25,2020-07-29,728.0,51.0,0,11


I will also remove Series 16 from the **regular_series** dataframe because its data only goes up to the eighth out of ten episodes in the series.

In [8]:
# Remove Series 16 due to incomplete data
regular_series = regular_series.drop(20)

KeyError: '[20] not found in axis'

Because none of the specials are included in this database, the column *special*, which indicates if the series was a special or not, is made redundant. Therefore, I will drop this column from the **regular_series** dataframe.

In [9]:
regular_series = regular_series.drop(columns='special')
regular_series

,id,name,episodes,champion,champion_label,air_start,air_end,studio_start,studio_end,points,tasks,TMI
0,1,Series 1,6,4.0,Josh Widdicombe,2015-07-28,2015-09-01,2015-03-23,2015-03-25,436.0,32.0,1
1,2,Series 2,5,11.0,Katherine Ryan,2016-06-21,2016-07-19,NaN,NaN,417.0,28.0,2
2,3,Series 3,5,16.0,Rob Beckett,2016-10-04,2016-11-01,NaN,NaN,386.0,27.0,3
3,4,Series 4,8,22.0,Noel Fielding,2017-04-25,2017-06-13,NaN,NaN,668.0,46.0,4
4,5,Series 5,8,24.0,Bob Mortimer,2017-09-13,2017-11-01,2017-07-03,2017-07-06,631.0,44.0,5
6,6,Series 6,10,35.0,Liza Tarbuck,2018-05-02,2018-07-04,2018-03-22,2018-03-28,837.0,57.0,7
7,7,Series 7,10,40.0,Kerry Godliman,2018-09-05,2018-11-07,2018-07-19,2018-07-25,816.0,56.0,8
8,8,Series 8,10,45.0,Lou Sanders,2019-05-08,2019-07-10,2019-03-21,2019-03-27,749.0,54.0,9
9,9,Series 9,10,49.0,Ed Gamble,2019-09-04,2019-11-06,2019-07-18,2019-07-24,766.0,51.0,10
10,10,Series 10,10,57.0,Richard Herring,2020-10-15,2020-12-17,2020-07-25,2020-07-29,728.0,51.0,11


It is also important to make sure any cells with "NaN" values are removed from the dataset. The rows for Series 2-4, 11, and 12 have NaN values under the *studio_start* and *studio_end* columns. As those rows are valuable for my research question, I will still keep them, but remove the *studio_start* and *studio_end* columns.

Additionally, I will remove the *air_start* and *air_end* columns, purely due to their redundancy to my research question.

All of these columns are removed using the .drop() function.

In [10]:
# Start by calling up the index values for the columns in 'regular_series'
pd.Series(list(regular_series.columns))

0                 id
1               name
2           episodes
3           champion
4     champion_label
5          air_start
6            air_end
7       studio_start
8         studio_end
9             points
10             tasks
11               TMI
dtype: object

In [11]:
regular_series = regular_series.drop(columns=['air_start', 'air_end', 'studio_start', 'studio_end'])
regular_series.head()

,id,name,episodes,champion,champion_label,points,tasks,TMI
0,1,Series 1,6,4.0,Josh Widdicombe,436.0,32.0,1
1,2,Series 2,5,11.0,Katherine Ryan,417.0,28.0,2
2,3,Series 3,5,16.0,Rob Beckett,386.0,27.0,3
3,4,Series 4,8,22.0,Noel Fielding,668.0,46.0,4
4,5,Series 5,8,24.0,Bob Mortimer,631.0,44.0,5


### Clean and transform 'Episodes' dataframe

The above dataset **regular_series** shows the winners of each regular series of Taskmaster. Now, let's look at the dataframe **episodes**, which includes who won each episode.

In [12]:
episodes.head(n=15)

,id,series,series_label,episode,title,winner,winner_label,air_date,studio_date,points,tasks,finale,TMI
0,1,1,Series 1,1,Melon buffet.,3,Frank Skinner,2015-07-28,2015-03-23,75,5,0,1
1,2,1,Series 1,2,The pie whisperer.,5,Roisin Conaty,2015-08-04,2015-03-23,78,5,0,2
2,3,1,Series 1,3,The poet and the egg.,4,Josh Widdicombe,2015-08-11,2015-03-24,84,6,0,3
3,4,1,Series 1,4,Down an octave.,4,Josh Widdicombe,2015-08-18,2015-03-24,58,5,0,4
4,5,1,Series 1,5,Little denim shorts.,3,Frank Skinner,2015-08-25,2015-03-25,78,6,0,5
5,6,1,Series 1,6,The last supper.,7,Tim Key,2015-09-01,2015-03-25,63,5,1,6
6,7,2,Series 2,1,Fear of failure.,12,Richard Osman,2016-06-21,NaN,72,5,0,7
7,8,2,Series 2,2,Pork is a sausage.,10,Jon Richardson,2016-06-28,NaN,90,6,0,8
8,9,2,Series 2,3,A pistachio eclair.,11,Katherine Ryan,2016-07-05,NaN,120,6,0,9
9,10,2,Series 2,4,Welcome to Rico Face.,8,Doc Brown,2016-07-12,NaN,67,6,0,10


In [13]:
episodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            146 non-null    int64 
 1   series        146 non-null    int64 
 2   series_label  146 non-null    object
 3   episode       146 non-null    int64 
 4   title         146 non-null    object
 5   winner        146 non-null    int64 
 6   winner_label  146 non-null    object
 7   air_date      146 non-null    object
 8   studio_date   106 non-null    object
 9   points        146 non-null    int64 
 10  tasks         146 non-null    int64 
 11  finale        146 non-null    int64 
 12  TMI           146 non-null    int64 
dtypes: int64(8), object(5)
memory usage: 15.0+ KB


Let's clean up this dataframe as well and include any changes in the new dataframe **episodes_filtered**. From calling the .info() function, I know that the only column containing null values is the *studio_date* column, which has 106 non-null values while the remaining columns have 146 non-null values. This column will be removed entirely rather than calling a function like .dropna() because I want to retain the remaining information included in the rows where those null values are found.

I also feel the air date and studio date information included in the eponymous columns is redundant for the question I am trying to answer. Additionally, the column titled *TMI* is redundant because its values are identical to the *id* column.

Therefore, the columns *air_date*, *studio_date*, and *TMI* are removed using the .drop() function.

In [14]:
episodes_filtered = episodes.drop(columns=['air_date', 'studio_date', 'TMI'])
episodes_filtered.head()

,id,series,series_label,episode,title,winner,winner_label,points,tasks,finale
0,1,1,Series 1,1,Melon buffet.,3,Frank Skinner,75,5,0
1,2,1,Series 1,2,The pie whisperer.,5,Roisin Conaty,78,5,0
2,3,1,Series 1,3,The poet and the egg.,4,Josh Widdicombe,84,6,0
3,4,1,Series 1,4,Down an octave.,4,Josh Widdicombe,58,5,0
4,5,1,Series 1,5,Little denim shorts.,3,Frank Skinner,78,6,0


### Merge 'regular_series' and 'episodes_filtered' dataframes

Now that the **regular_series** and **episodes_filtered** datasets have been created and cleaned, let's have another look at them to determine how to illustrate how many episodes have been won by the series winner.

In [15]:
regular_series.head()

,id,name,episodes,champion,champion_label,points,tasks,TMI
0,1,Series 1,6,4.0,Josh Widdicombe,436.0,32.0,1
1,2,Series 2,5,11.0,Katherine Ryan,417.0,28.0,2
2,3,Series 3,5,16.0,Rob Beckett,386.0,27.0,3
3,4,Series 4,8,22.0,Noel Fielding,668.0,46.0,4
4,5,Series 5,8,24.0,Bob Mortimer,631.0,44.0,5


In [16]:
episodes_filtered.head()

,id,series,series_label,episode,title,winner,winner_label,points,tasks,finale
0,1,1,Series 1,1,Melon buffet.,3,Frank Skinner,75,5,0
1,2,1,Series 1,2,The pie whisperer.,5,Roisin Conaty,78,5,0
2,3,1,Series 1,3,The poet and the egg.,4,Josh Widdicombe,84,6,0
3,4,1,Series 1,4,Down an octave.,4,Josh Widdicombe,58,5,0
4,5,1,Series 1,5,Little denim shorts.,3,Frank Skinner,78,6,0


The best way to use these two dataframes to determine how many Taskmaster episodes were won by that series' winner is a two-step process of merging the **regular_series** and **episodes_filtered** dataframes together and then aggregating the resulting dataframe. The merge will be done using a left merge preserving the rows of the **episodes_filtered** dataframe (ie. perform a left merge with **episodes_filtered** as the left-side dataframe and **regular_series** as the right-side dataframe). The merge will be served to the variable **episode_winners**.

After additional data cleaning, the .agg() and .groupby() methods will be performed on **episode_winners** to apply aggregate functions.

In [17]:
episode_winners = pd.merge(left=episodes_filtered,
                          right=regular_series,
                          left_on=['series_label'],
                          right_on=['name'],
                          how='left')
episode_winners.head(n=15)

,id_x,series,series_label,episode,title,winner,winner_label,points_x,tasks_x,finale,id_y,name,episodes,champion,champion_label,points_y,tasks_y,TMI
0,1,1,Series 1,1,Melon buffet.,3,Frank Skinner,75,5,0,1.0,Series 1,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
1,2,1,Series 1,2,The pie whisperer.,5,Roisin Conaty,78,5,0,1.0,Series 1,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
2,3,1,Series 1,3,The poet and the egg.,4,Josh Widdicombe,84,6,0,1.0,Series 1,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
3,4,1,Series 1,4,Down an octave.,4,Josh Widdicombe,58,5,0,1.0,Series 1,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
4,5,1,Series 1,5,Little denim shorts.,3,Frank Skinner,78,6,0,1.0,Series 1,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
5,6,1,Series 1,6,The last supper.,7,Tim Key,63,5,1,1.0,Series 1,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
6,7,2,Series 2,1,Fear of failure.,12,Richard Osman,72,5,0,2.0,Series 2,5.0,11.0,Katherine Ryan,417.0,28.0,2.0
7,8,2,Series 2,2,Pork is a sausage.,10,Jon Richardson,90,6,0,2.0,Series 2,5.0,11.0,Katherine Ryan,417.0,28.0,2.0
8,9,2,Series 2,3,A pistachio eclair.,11,Katherine Ryan,120,6,0,2.0,Series 2,5.0,11.0,Katherine Ryan,417.0,28.0,2.0
9,10,2,Series 2,4,Welcome to Rico Face.,8,Doc Brown,67,6,0,2.0,Series 2,5.0,11.0,Katherine Ryan,417.0,28.0,2.0


Clean up **episode_winners** data by dropping the *id_y* and *name* columns

In [18]:
episode_winners = episode_winners.drop(columns=['id_y', 'name'])
episode_winners.head()

,id_x,series,series_label,episode,title,winner,winner_label,points_x,tasks_x,finale,episodes,champion,champion_label,points_y,tasks_y,TMI
0,1,1,Series 1,1,Melon buffet.,3,Frank Skinner,75,5,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
1,2,1,Series 1,2,The pie whisperer.,5,Roisin Conaty,78,5,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
2,3,1,Series 1,3,The poet and the egg.,4,Josh Widdicombe,84,6,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
3,4,1,Series 1,4,Down an octave.,4,Josh Widdicombe,58,5,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
4,5,1,Series 1,5,Little denim shorts.,3,Frank Skinner,78,6,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0


Clean up **episode_winners** data by renaming *id_x* as *id*, *winner* as *ep_winner*, *winner_label* as *ep_winner_label*, *points_x* as *ep_points*, *tasks_x* as *ep_tasks*, *points_y* as *series_points*, *tasks_y* as *series_tasks*

In [19]:
episode_winners = episode_winners.rename(columns={'id_x': 'id',
                                          'winner': 'ep_winner',
                                          'winner_label': 'ep_winner_label',
                                          'points_x': 'ep_points',
                                          'tasks_x': 'ep_tasks',
                                          'points_y': 'series_points',
                                          'tasks_y': 'series_tasks'})
episode_winners.head()

,id,series,series_label,episode,title,ep_winner,ep_winner_label,ep_points,ep_tasks,finale,episodes,champion,champion_label,series_points,series_tasks,TMI
0,1,1,Series 1,1,Melon buffet.,3,Frank Skinner,75,5,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
1,2,1,Series 1,2,The pie whisperer.,5,Roisin Conaty,78,5,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
2,3,1,Series 1,3,The poet and the egg.,4,Josh Widdicombe,84,6,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
3,4,1,Series 1,4,Down an octave.,4,Josh Widdicombe,58,5,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0
4,5,1,Series 1,5,Little denim shorts.,3,Frank Skinner,78,6,0,6.0,4.0,Josh Widdicombe,436.0,32.0,1.0


Clean up **episode_winners** by performing a type conversion so the float64 values of the *episodes*, *champion*, *series_points*, *series_tasks*, and *TMI* columns become int64 values like *id*, *series*, *episode*, *ep_winner*, *ep_points*, *ep_tasks*, and *finale*.

In [20]:
display(episode_winners.dtypes)

id                   int64
series               int64
series_label        object
episode              int64
title               object
ep_winner            int64
ep_winner_label     object
ep_points            int64
ep_tasks             int64
finale               int64
episodes           float64
champion           float64
champion_label      object
series_points      float64
series_tasks       float64
TMI                float64
dtype: object

In [21]:
episode_winners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               146 non-null    int64  
 1   series           146 non-null    int64  
 2   series_label     146 non-null    object 
 3   episode          146 non-null    int64  
 4   title            146 non-null    object 
 5   ep_winner        146 non-null    int64  
 6   ep_winner_label  146 non-null    object 
 7   ep_points        146 non-null    int64  
 8   ep_tasks         146 non-null    int64  
 9   finale           146 non-null    int64  
 10  episodes         132 non-null    float64
 11  champion         132 non-null    float64
 12  champion_label   132 non-null    object 
 13  series_points    132 non-null    float64
 14  series_tasks     132 non-null    float64
 15  TMI              132 non-null    float64
dtypes: float64(5), int64(7), object(4)
memory usage: 18.4+ KB


NaN (Not a Number) values cannot undergo type conversion, so any rows with non-null values must be removed from the **episode_winners** dataframe. I want to see which rows have NaN values, so I have created a new dataframe **winners_isna** which uses the .isna() and .any() functions to pull the NaN rows from **episode_winners**.

In [22]:
winners_isna = episode_winners[episode_winners.isna().any(axis=1)]
winners_isna

,id,series,series_label,episode,title,ep_winner,ep_winner_label,ep_points,ep_tasks,finale,episodes,champion,champion_label,series_points,series_tasks,TMI
32,33,-1,CoC,1,Wiley giraffe blower.,30,Katherine Ryan,85,5,0,NaN,NaN,NaN,NaN,NaN,NaN
33,34,-1,CoC,2,I've sinned again.,29,Josh Widdicombe,79,5,0,NaN,NaN,NaN,NaN,NaN,NaN
84,85,-2,NYT 2021,1,The fastest duck.,62,Shirley Ballas,62,5,0,NaN,NaN,NaN,NaN,NaN,NaN
105,106,-3,NYT 2022,1,Basic recipe 28.,73,Adrian Chiles,68,5,0,NaN,NaN,NaN,NaN,NaN,NaN
116,117,-4,CoC II,1,The Alpine Darling.,87,Richard Herring,66,5,0,NaN,NaN,NaN,NaN,NaN,NaN
127,128,-5,NYT 2023,1,That's a swizz.,96,Mo Farah,76,5,0,NaN,NaN,NaN,NaN,NaN,NaN
138,139,16,Series 16,1,The natural friends.,105,Sam Campbell,80,5,0,NaN,NaN,NaN,NaN,NaN,NaN
139,140,16,Series 16,2,Hell is here.,106,Sue Perkins,76,5,0,NaN,NaN,NaN,NaN,NaN,NaN
140,141,16,Series 16,3,Languidly.,105,Sam Campbell,76,5,0,NaN,NaN,NaN,NaN,NaN,NaN
141,142,16,Series 16,4,Dynamite chicks.,106,Sue Perkins,81,5,0,NaN,NaN,NaN,NaN,NaN,NaN


All of the rows with NaN values are either the special episodes (New Years Treat and Champion of Champions) or the episodes from Series 16, which, at the time of writing, has not been updated to feature all the episodes from that series. The NaN values are also from columns taken from **regular_series** during the merge with **episodes_filtered**, which already had those rows removed to focus on regular episodes from completed series. Therefore, I will go ahead and call .dropna() to remove these rows as they are redundant to the research question.

In [23]:
episode_winners = episode_winners.dropna()

In [24]:
episode_winners = episode_winners.astype({'episodes':'int',
                                        'champion':'int',
                                        'series_points':'int',
                                        'series_tasks':'int',
                                        'TMI':'int'})
episode_winners.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132 entries, 0 to 137
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               132 non-null    int64 
 1   series           132 non-null    int64 
 2   series_label     132 non-null    object
 3   episode          132 non-null    int64 
 4   title            132 non-null    object
 5   ep_winner        132 non-null    int64 
 6   ep_winner_label  132 non-null    object
 7   ep_points        132 non-null    int64 
 8   ep_tasks         132 non-null    int64 
 9   finale           132 non-null    int64 
 10  episodes         132 non-null    int32 
 11  champion         132 non-null    int32 
 12  champion_label   132 non-null    object
 13  series_points    132 non-null    int32 
 14  series_tasks     132 non-null    int32 
 15  TMI              132 non-null    int32 
dtypes: int32(5), int64(7), object(4)
memory usage: 15.0+ KB


### How many episodes did each series champion win?

Now that the dataframe **episode_winners** has been created and cleaned of any null and inappropriate data, I can use the prepared data to answer the question of *How many Taskmaster episodes were won by the champion for that series?*

To start, I filter the **episode_winners** dataframe to only include rows where the *ep_winner_label* value is equal to the *champion_label* value. This filter is saved under the new variable **series_winners_ep**.

In [26]:
series_winners_ep = episode_winners[episode_winners['ep_winner_label'] == episode_winners['champion_label']]
series_winners_ep.head()

,id,series,series_label,episode,title,ep_winner,ep_winner_label,ep_points,ep_tasks,finale,episodes,champion,champion_label,series_points,series_tasks,TMI
2,3,1,Series 1,3,The poet and the egg.,4,Josh Widdicombe,84,6,0,6,4,Josh Widdicombe,436,32,1
3,4,1,Series 1,4,Down an octave.,4,Josh Widdicombe,58,5,0,6,4,Josh Widdicombe,436,32,1
8,9,2,Series 2,3,A pistachio eclair.,11,Katherine Ryan,120,6,0,5,11,Katherine Ryan,417,28,2
12,13,3,Series 3,2,The dong and the gong.,16,Rob Beckett,85,6,0,5,16,Rob Beckett,386,27,3
15,16,3,Series 3,5,The F.I.P.,16,Rob Beckett,67,5,1,5,16,Rob Beckett,386,27,3


The new variable **series_winners_ep** then is grouped by the *champion_label* column, accompanied by the *series* and *episodes* columns. The .agg() function is performed with the counts for *ep_winner_label*. This .groupby().agg() call is set to the variable **winners**.

In [27]:
winners = series_winners_ep.groupby(['champion_label','series','episodes']).agg({'ep_winner_label':'count'})
winners

,,,ep_winner_label
champion_label,series,episodes,
Bob Mortimer,5,8,3
Dara Ó Briain,14,10,3
Ed Gamble,9,10,4
Josh Widdicombe,1,6,2
Katherine Ryan,2,5,1
Kerry Godliman,7,10,3
Liza Tarbuck,6,10,2
Lou Sanders,8,10,3
Mae Martin,15,10,3


Now that we have the number of episodes won by each series' champion, I want to re-order the dataframe **winners** to follow the series order, not alphabetical order. Below is the final product showing the number of episodes each series champion won in series order.

In [28]:
winners = winners.sort_values('series')
winners

,,,ep_winner_label
champion_label,series,episodes,
Josh Widdicombe,1,6,2
Katherine Ryan,2,5,1
Rob Beckett,3,5,2
Noel Fielding,4,8,1
Bob Mortimer,5,8,3
Liza Tarbuck,6,10,2
Kerry Godliman,7,10,3
Lou Sanders,8,10,3
Ed Gamble,9,10,4


## Profanity spoken by the Taskmaster

In typical British humour, *Taskmaster* is filled with profanities. The Taskmaster himself, Greg Davies, is not immune to this. Using the **profanity** dataframe, I determine the kinds of profanities the Taskmaster uses, and which one is his favourite.

### Clean and transform 'Profanity' dataframe

Let's inspect the **profanity** dataframe. This dataframe lists every moment a profanity is used in *Taskmaster* in order. It features the series number, episode number, the task at hand, the speaker, the quote featuring the profanity as well as the profanity itself (*roots* column). If more than one profanity is used in a single sentence, they are listed together in the *roots* column.

In [29]:
profanity.head()

,id,series,series_label,episode,episode_label,task,task_label,speaker,speaker_label,roots,quote,studio
0,1,1,Series 1,1,Melon buffet.,1.0,The most unusual item,6,Romesh Ranganathan,"[""shit""]",I think we can all agree it's the shittest pre...,1
1,2,1,Series 1,1,Melon buffet.,1.0,The most unusual item,6,Romesh Ranganathan,"[""piss""]",You know the granules? The snow density is pis...,1
2,3,1,Series 1,1,Melon buffet.,1.0,The most unusual item,2,Greg Davies,"[""shit""]",And the winner of this competition will have t...,1
3,4,1,Series 1,1,Melon buffet.,2.0,Eat as much watermelon as possible,6,Romesh Ranganathan,"[""shit""]","And then when I threw it I was like, ""Holy shi...",1
4,5,1,Series 1,1,Melon buffet.,2.0,Eat as much watermelon as possible,2,Greg Davies,"[""shit""]",I've got an image of you at an all-you-can-eat...,1


In [30]:
profanity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1045 non-null   int64  
 1   series         1045 non-null   int64  
 2   series_label   1045 non-null   object 
 3   episode        1045 non-null   int64  
 4   episode_label  1045 non-null   object 
 5   task           1033 non-null   float64
 6   task_label     1033 non-null   object 
 7   speaker        1045 non-null   int64  
 8   speaker_label  1045 non-null   object 
 9   roots          1045 non-null   object 
 10  quote          1045 non-null   object 
 11  studio         1045 non-null   int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 98.1+ KB


I am only curious about the profanities produced by the Taskmaster, Greg Davies. Therefore, I create a new variable **taskmaster_profanity**, which contains the filtered **profanity** dataframe containing only profanities produced by Greg Davies.

In [30]:
taskmaster_profanity = profanity[profanity['speaker_label'] == 'Greg Davies']
taskmaster_profanity.head()

,id,series,series_label,episode,episode_label,task,task_label,speaker,speaker_label,roots,quote,studio
2,3,1,Series 1,1,Melon buffet.,1.0,The most unusual item,2,Greg Davies,"[""shit""]",And the winner of this competition will have t...,1
4,5,1,Series 1,1,Melon buffet.,2.0,Eat as much watermelon as possible,2,Greg Davies,"[""shit""]",I've got an image of you at an all-you-can-eat...,1
9,10,1,Series 1,1,Melon buffet.,3.0,Paint a horse while riding a horse,2,Greg Davies,"[""shit""]",The top three are absolutely shit.,1
13,14,1,Series 1,1,Melon buffet.,6.0,Find Alex,2,Greg Davies,"[""hell""]","Alex, what the hell are we gonna do about this?",1
20,21,1,Series 1,2,The pie whisperer.,8.0,High-five a 55-year-old,2,Greg Davies,"[""Christ"",""shit""]","I mean, Jesus Christ, I thought you'd only sto...",1


## Split rows to show unique profanities

As stated previously, multiple profanities in a single quote are listed together in one row in the **profanity** and **taskmaster_profanity** dataframes. I create a new variable, **taskmaster_exploded**, which includes the exploded *roots* column so each individual profanity is listed separately.

In [31]:
taskmaster_exploded = taskmaster_profanity['roots'].str.split(',').explode().reset_index(drop=True)

In [32]:
taskmaster_exploded.head(n=10)

0      ["shit"]
1      ["shit"]
2      ["shit"]
3      ["hell"]
4     ["Christ"
5       "shit"]
6    ["Christ"]
7    ["Christ"]
8      ["fuck"]
9      ["fuck"]
Name: roots, dtype: object

In [33]:
taskmaster_exploded.info()

<class 'pandas.core.series.Series'>
RangeIndex: 344 entries, 0 to 343
Series name: roots
Non-Null Count  Dtype 
--------------  ----- 
344 non-null    object
dtypes: object(1)
memory usage: 2.8+ KB


To make the data more presentable, I strip the [] and "" characters from the **taskmaster_exploded** dataframe.

In [34]:
taskmaster_exploded = taskmaster_exploded.str.strip('[]')
taskmaster_exploded

0          "shit"
1          "shit"
2          "shit"
3          "hell"
4        "Christ"
          ...    
339        "fuck"
340    "ass/arse"
341        "shit"
342        "piss"
343        "piss"
Name: roots, Length: 344, dtype: object

In [35]:
taskmaster_exploded = taskmaster_exploded.str.strip('""')
taskmaster_exploded

0          shit
1          shit
2          shit
3          hell
4        Christ
         ...   
339        fuck
340    ass/arse
341        shit
342        piss
343        piss
Name: roots, Length: 344, dtype: object

From this, I can list the unique profanities produced by Taskmaster Greg Davies with their unique counts in descending order using .value_counts().

In [36]:
taskmaster_exploded.value_counts()

roots
fuck        108
shit         94
Christ       39
hell         25
piss         21
dick         19
prick        14
ass/arse      8
bastard       5
tits          3
cock          2
wank          1
cunt          1
bugger        1
bollock       1
pussy         1
twat          1
Name: count, dtype: int64

# How many egg/duck tasks?

Two reoccuring focii of *Taskmaster* are eggs and rubber ducks. What is the percentage of tasks which feature eggs? What is the percentage of tasks which feature rubber ducks?

I start by inspecting the dataframe **task_briefs**.

In [37]:
task_briefs.head(n=10)

,id,task,task_label,brief
0,1,1,The most unusual item,Bring in the most unusual item.
1,2,2,Eat as much watermelon as possible,In the lab there is a watermelon.\nEat as much...
2,3,3,Paint a horse while riding a horse,"Paint the best picture of a horse, whilst ridi..."
3,4,4,Empty the bathtub,Completely empty this bathtub.\nFastest wins.\...
4,5,5,Pop up a tent and pop on a onesie,Pop up a tent.\nGet in the tent.\nZip up the t...
5,6,6,Find Alex,Find Alex.\nFastest wins.\nYour time starts now.
6,7,7,The most impressive item,Bring in your most impressive item.
7,8,8,High-five a 55-year-old,High-five a 55-year-old.\nFastest wins.\nYour ...
8,9,9,Identify the contents of the pies,Identify the contents of these pies.\nYou may ...
9,10,10,Do something that will look impressive in reverse,Do something that will look impressive in reve...


In [38]:
task_briefs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809 entries, 0 to 808
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          809 non-null    int64 
 1   task        809 non-null    int64 
 2   task_label  809 non-null    object
 3   brief       809 non-null    object
dtypes: int64(2), object(2)
memory usage: 25.4+ KB


The **task_briefs** dataframe lists 809 tasks with their full description. The dataframe does not feature the series and episode numbers/titles that each task appears in, so, unlike previous dataframes, I cannot filter them by completed vs. non-completed series or regular series vs. specials.

*Recall that the dataframe **tasks** includes columns for the episode and series numbers and labels, but this is an empty dataframe, so I cannot use it.*

First, I filter the **task_briefs** dataframe to only list tasks which include the case-insensitive string 'egg' under the *brief* column. This filter is saved to the variable **egg_tasks**.

In [39]:
egg_tasks = task_briefs[task_briefs['brief'].str.contains('egg',case=False)]
egg_tasks.head(n=10)

,id,task,task_label,brief
15,16,16,Get an egg as high as possible,"Using only the items currently on this table, ..."
23,24,24,Fill an eggcup with tears,Fill an egg cup with as many tears as possible...
33,34,34,Play mini golf with eggs,Get the lowest golf score using eggs.\nYou mus...
77,78,78,Fill an eggcup with sweat,Fill this eggcup with your own sweat.\nMost sw...
102,103,103,Put an egg in an egg cup without touching either,"Without touching the egg or the eggcup, get th..."
193,194,194,Throw an egg through a basketball hoop,Throw your egg through the hoop and catch it a...
214,215,215,Take a picture of people holding eggs,"Take a picture of a group of people, each hold..."
231,232,232,Remove the tablecloth from underneath the eggs,Pull this tablecloth off this table.\nMost egg...
234,235,235,Throw eggs onto a shelf,Throw as many eggs onto the shelf as possible....
246,247,247,Roll eggs to the centre of the target,Get an egg as close to the centre of the targe...


In [40]:
egg_tasks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 15 to 773
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          26 non-null     int64 
 1   task        26 non-null     int64 
 2   task_label  26 non-null     object
 3   brief       26 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.0+ KB


In [41]:
duck_tasks = task_briefs[task_briefs['brief'].str.contains('duck',case=False)]
duck_tasks.head(n=10)

,id,task,task_label,brief
96,97,97,Fell all the ducks,Fell all the rubber ducks.\nFastest wins.\nYou...
143,144,144,Do something surprising with a duck,Do something surprising with this rubber duck....
144,145,145,Draw the median duck,Draw the median duck.\nThe median duck alone w...
237,238,238,Knock over the ducks without leaving the carpet,Knock over as many ducks as possible.\nYou mus...
325,326,326,Sneak up on Alex,Alex is on the bridge in the distance with lig...
377,378,378,Hook a duck while wearing inversion goggles,Correctly wearing these goggles and with your ...
586,587,587,Duck-slide,Get the highest score by sliding your ducks.\n...
598,599,599,Find all ten ducks in the lab,Find all 10 ducks.\nYou must stay in the lab.\...
610,611,611,Enable Alex to bite his duck,Enable Alex to bite his duck.\nYou may not enc...
623,624,624,Pop balls out of your hole,Pop three red balls out of your hole.\nYou may...


In [42]:
duck_tasks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 96 to 769
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          16 non-null     int64 
 1   task        16 non-null     int64 
 2   task_label  16 non-null     object
 3   brief       16 non-null     object
dtypes: int64(2), object(2)
memory usage: 640.0+ bytes


Among the 809 tasks included in the dataframe **task_briefs**, there are 26 which feature eggs, as shown in **egg_tasks**, and 16 which feature rubber ducks, as shown in **duck_tasks**.

# Conclusions and Comments

## Findings

For this analysis of **A Ridiculously Comprehensive Taskmaster Database**, I focussed on three questions to be answered using the available dataframes:
1. How many episodes were won by the champion for that series?
2. What profanities are used by Taskmaster Greg Davies?
3. How many tasks feature eggs? How many feature rubber ducks?

The first question on episodes won by series champions was investigated using the dataframes **series** and **episodes**. They were each cleaned and filtered to include only episodes from completed, regular series, resulting in the new dataframes titled **regular_series** and **episodes_filtered**. Then, a left merge is performed, retaining the columns of the **regular_series** dataframe while adding the **episodes_filtered** dataframe, merging the **series_label** column from the former with the **name** column from the latter.

From that merge, after additional cleaning and transforming is performed on the new dataframe, **episode_winners**, it then was filtered for just the rows where the values of the columns *ep_winner_label* and *champion_label* were equal (ex. "Josh Widdicome" is the value of both columns). This filter was saved to the new varible name **series_winners_ep**.

Finally, the **series_winners_ep** underwent the .groupby() and .agg() functions to give the counts for *ep_winner_label* for each unique name in *champion_label* (in other words, the counts for each individual champion in the dataframe). The functions were saved under the variable name **winners**.

The second question on what profanities are used by the Taskmaster used the **profanity** dataframe. It was filtered to include only profanities produced by Greg Davies using the filter command *speaker_label* == *'Greg Davies'*, which was saved to the variable **taskmaster_profanity**. Then, the *roots* column was exploded to separate instances where more than one profanity was included in a single statement. This was saved to the variable **taskmaster_exploded**. This final dataframe then was sorted using the .value_counts() command to give the unique counts for each profanity used by the Taskmaster.

The final question on the number of tasks which feature eggs and rubber ducks used the **task_briefs** dataframe. For each query, the dataframe was filtered for the string 'egg' and 'duck', respectively, using the .contains() function, with *case* set as 'False' so that the results were not case sensitive. These filteres were saved to the variable names **egg_tasks** and **duck_tasks**, respectively. Their results showed that, of the 809 tasks included in the **task_briefs** dataframe, 26 tasks featured eggs and 16 featured ducks. 

## Further Research

As is often the case, new questions arose from the research I performed on the Taskmaster database. Some questions were not addressed in this analysis because there was not enough information available.

For the first question, *How many episodes were won by the champion for that series?*, I could have additionally calculated the percentages of episodes won by winners. This would have normalized the results as the first five series range in five to eight episodes in length, while the series since are 10 episodes each. I also could have looked into the episodes won by non-series champions and compared the two against one another. Are there series contestants that were more successful at winning episodes than that series' champion?

For the second question, *What profanities are used by Taskmaster Greg Davies?*, another perspective would have been to filter for each individual series and compare the Taskmaster's profanity use to the contestants for each series. Does he tend to use more or less profanity than the contestants? Or, just looking at the contestants, how much profanity is used in the pre-filmed tasks versus in the studio? What degree of profanity is used by the Taskmaster's Assistant, Little Alex Horne?

For the third and final question, *How many tasks feature eggs? How many feature rubber ducks?*, the data was hindered by the **task_briefs** dataframe which was used to answer those questions. This dataframe did not include information about the series and episode each task appeared in, so I could not filter for special versus regular episodes, or completed versus non-completed series. It would have been interesting to see the distribution of egg and duck tasks throughout the entire run of the show. Which series featured the most egg/duck appearances? Did these tasks appear more frequently early on in the show and then lessen over the course of the show, or vice versa?